I created a new kind of mpohamiltonian for quantum chemistry hamiltonians, because you can gain some speed there by pre-summing the environments. I also invested a bunch of effort making the effective hamiltonian fast. In this notebook I try to benchmark the MPSKit effective hamiltonians against my (?optimized) implementation.

I don't make use of the pre-summation in this benchmark, both codes should do the exact same thing.

In [1]:
using Revise, MPSKit, MPSKitModels, TensorKit
using LinearAlgebra, Base.Threads, BenchmarkTools
using MPSKitExperimental:FusedMPOHamiltonian

In [2]:
BLAS.set_num_threads(1)

In [3]:
nthreads()

4

## heisenberg spin 1

In [8]:
fmps_len = 10;
middle_site = Int(round(fmps_len/2))

ts = FiniteMPS(rand,ComplexF64,fmps_len,Rep[SU₂](1=>1),Rep[SU₂](i => 20 for i in 0:10));

th_orig = heisenberg_XXX(SU2Irrep);
th_fused = convert(FusedMPOHamiltonian,repeat(th_orig,length(ts)));

env_orig = environments(ts,th_orig);
env_fused = environments(ts,th_fused);

ac = ts.AC[middle_site]
ac_eff_orig = MPSKit.∂∂AC(middle_site,ts,th_orig,env_orig);
ac_eff_fused = MPSKit.∂∂AC(middle_site,ts,th_fused,env_fused);


ac2 = ts.AC[middle_site]*MPSKit._transpose_tail(ts.AR[middle_site+1])
ac2_eff_orig = MPSKit.∂∂AC2(middle_site,ts,th_orig,env_orig);
ac2_eff_fused = MPSKit.∂∂AC2(middle_site,ts,th_fused,env_fused);

In [9]:
@btime ac_eff_orig(ac);

  862.044 μs (14410 allocations: 1.43 MiB)


In [10]:
@btime ac_eff_fused(ac);

  33.170 μs (971 allocations: 123.69 KiB)


In [11]:
@btime ac2_eff_orig(ac2);

  1.313 ms (35055 allocations: 3.55 MiB)


In [12]:
@btime ac2_eff_fused(ac2);

  40.050 μs (1003 allocations: 75.84 KiB)


## some weird hamiltonian

In [20]:
using MAT
include("haagerup/H3_anyon.jl")

In [23]:
fmps_len = 10;
middle_site = Int(round(fmps_len/2))

physical = Vect[H3](H3(4)=>1);
virtual = Vect[H3](sector => 20 for sector in TensorKit.SectorValues{H3}());
ts = FiniteMPS(rand,ComplexF64,fmps_len,physical,virtual);

mpotensor = TensorMap(ones,ComplexF64,physical*physical,physical*physical);
blocks(mpotensor)[H3(1)]*=0;
blocks(mpotensor)[H3(5)]*=0;
blocks(mpotensor)[H3(6)]*=0;
th_orig = MPOHamiltonian(-mpotensor);
th_fused = convert(FusedMPOHamiltonian,repeat(th_orig,length(ts)));

env_orig = environments(ts,th_orig);
env_fused = environments(ts,th_fused);

ac = ts.AC[middle_site]
ac_eff_orig = MPSKit.∂∂AC(middle_site,ts,th_orig,env_orig);
ac_eff_fused = MPSKit.∂∂AC(middle_site,ts,th_fused,env_fused);


ac2 = ts.AC[middle_site]*MPSKit._transpose_tail(ts.AR[middle_site+1])
ac2_eff_orig = MPSKit.∂∂AC2(middle_site,ts,th_orig,env_orig);
ac2_eff_fused = MPSKit.∂∂AC2(middle_site,ts,th_fused,env_fused);

In [24]:
@btime ac_eff_orig(ac);

  3.226 ms (52566 allocations: 6.93 MiB)


In [25]:
@btime ac_eff_fused(ac);

  456.387 μs (1703 allocations: 405.88 KiB)


In [26]:
@btime ac2_eff_orig(ac2);

  12.658 ms (324728 allocations: 33.43 MiB)


In [27]:
@btime ac2_eff_fused(ac2);

  828.195 μs (4019 allocations: 377.53 KiB)
